In [1]:
import os
os.chdir("..")

In [2]:
import json, requests

In [3]:
import pandas as pd
import numpy as np

from src.scrub import get_clean_iris
from src.deploy import generate_random_iris
from src.deploy import get_prediction

# Stream Prediction
---

- define the url where the api is served
- set up sample payload
- send a `post` request with the payload to the api
- decode the returned json into a dict
- change the payload and re-run a few times

In [7]:
url = "http://127.0.0.1:5000/api"

In [11]:
payload = json.dumps({
    'sl': 3,
    'sw': 5,
    'pl': 2.75,
    'pw': 2.1,
    'algo': 'dt'
})

In [12]:
r = requests.post(url, payload)

In [13]:
json.loads(r.text)

{'iris_type': 'virginica', 'model': 'dt'}

# Batch Predictions
---

- we simulate some real-world data by adding noise to actual data
- rename columns so they are consistent with what the api expects
- define a function to retrieve a prediction
- map the function over the rows

In [14]:
df = generate_random_iris()

- Test with a sample case

In [19]:
test_case = df.sample(1).iloc[0]
print(test_case)

get_prediction(test_case, 'dt', url=url)

sl    6.33
sw    3.16
pl    5.84
pw    1.49
Name: 0.75, dtype: float64


{'iris_type': 'virginica', 'model': 'dt'}

- Run over all cases

In [20]:
df['predicted_iris_type'] = \
(df
 .apply(lambda r: get_prediction(r, 'lr', url=url).get('iris_type'), 
        axis=1)
)

In [21]:
df.head()

,sl,sw,pl,pw,predicted_iris_type
0.75,7.53,3.96,4.41,3.09,virginica
0.75,5.91,1.86,3.85,1.11,versicolor
0.75,6.94,0.87,5.87,1.18,versicolor
0.75,5.24,0.52,2.94,1.30,versicolor
0.75,6.06,2.70,5.34,2.94,virginica
